In [1]:
import cx_Oracle
import pandas as pd

In [2]:
def connect_to_DB (username,password,hostname):
    """ Returns a connection to Oracle database"""
    try:
        connection = cx_Oracle.connect(username, password, hostname, encoding="UTF-8")
        print  ("Successffuly connected to the database")
    except:
        raise Exception('Connection failed! Please verifiy your login parameters')

    return connection

In [ ]:
# db connection parameters
hostname = 'bcgw.bcgov/idwprod1.bcgov'
username = input("Enter your BCGW username:")
password = input("Enter your BCGW password:")

In [4]:
# connect to BCGW
connection = connect_to_DB (username,password,hostname)

Successffuly connected to the database


In [13]:
#SQL to execute
query = """
          SELECT*
           
          FROM WHSE_TANTALIS.TA_CROWN_TENURES_SVW b

          WHERE SDO_ANYINTERACT (b.SHAPE, SDO_GEOMETRY(:wktvar, 3005)) = 'TRUE'
       """

In [14]:
# Execute query and read results into df
def read_query(connection,query,bind_vars):
    cursor = connection.cursor()
    try:
        cursor.execute(query, bind_vars)
        names = [ x[0] for x in cursor.description]
        rows = cursor.fetchall()
        return pd.DataFrame( rows, columns=names)
    
    finally:
        if cursor is not None:
            cursor.close()

In [15]:
# WKT string
wkt = 'MULTIPOLYGON (((1100843.5229000002 421022.85319999978, 1103224.7775999997 410545.33220000006,\
                      1116797.9298 404274.69470000081, 1132514.2112000007 408799.07870000042, \
                      1127275.4506999999 422372.23090000078, 1100843.5229000002 421022.85319999978)))'

In [16]:
# Define bind variables
bind_vars = {'wktvar': wkt}

In [17]:
df = read_query(connection,query,bind_vars)
df.head()

,INTRID_SID,TENURE_STAGE,TENURE_STATUS,TENURE_TYPE,TENURE_SUBTYPE,TENURE_PURPOSE,TENURE_SUBPURPOSE,CROWN_LANDS_FILE,APPLICATION_TYPE_CDE,TENURE_DOCUMENT,...,TENURE_AREA_IN_HECTARES,RESPONSIBLE_BUSINESS_UNIT,DISPOSITION_TRANSACTION_SID,CODE_CHR_STAGE,FEATURE_CODE,SHAPE,FEATURE_AREA_SQM,FEATURE_LENGTH_M,OBJECTID,SE_ANNO_CAD_DATA
0,153741,TENURE,DISPOSITION IN GOOD STANDING,RESERVE/NOTATION,NOTATION OF INTEREST,"ENVIRONMENT, CONSERVATION, & RECR",FISH AND WILDLIFE MANAGEMENT,1407024,NEW,R920030,...,0.025,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,160568,T,FL98000700,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at -0x69b...,225.1476,60.0198,1513,None
1,946268,TENURE,DISPOSITION IN GOOD STANDING,LICENCE,LICENCE OF OCCUPATION,QUARRYING,CONSTRUCTION STONE,1407245,REP,V921238,...,25.030,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,921238,T,FL98000500,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at -0x69b...,248988.4379,1995.2360,46856,None
